In [1]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pickle
import tensorflow.keras as keras


2023-11-11 08:30:58.848345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Import
train_df = tf.keras.utils.image_dataset_from_directory("archive/train",seed=45,image_size=(32, 32),batch_size=64)
test_df = tf.keras.utils.image_dataset_from_directory("archive/test",seed=123,image_size=(32, 32),batch_size=64)


Found 100000 files belonging to 2 classes.


2023-11-11 08:31:11.793268: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-11 08:31:11.793356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 20000 files belonging to 2 classes.


In [3]:
# model = tf.keras.applications.NASNetLarge(
#     input_shape=None,
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )


#model = keras.applications.NASNetMobile(weights=None, input_shape=(32, 32, 3), classes=7)
layers = []
layers.append(tf.keras.layers.Rescaling(1./255)) # Normalise pixel values
layers.append(tf.keras.layers.Conv2D(32, 3, activation='relu'))
layers.append(tf.keras.layers.MaxPooling2D())
layers.append(tf.keras.layers.Flatten())

# Building the ANN
layers.append(tf.keras.layers.Dense(64, activation='relu'))
layers.append(tf.keras.layers.Dense(32, activation='relu'))
layers.append(tf.keras.layers.Dense(32, activation='softmax'))


layers.append(tf.keras.layers.Dense(1, activation='sigmoid'))
layers.append(tf.keras.layers.Dense(2, activation='softmax'))


# Create and compile the model from layers
model = tf.keras.Sequential(layers)
model.compile(
  optimizer='Adamax',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Build the model so we can see a summary
model.build(input_shape=(None, 32, 32, 3))
model.summary()

# model = 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 64)                460864    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [4]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [5]:
history = model.fit(
  train_df,
  validation_data=test_df,
  epochs=10,
  verbose=1
)


Epoch 1/10


2023-11-11 08:32:35.235021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [100000]
	 [[{{node Placeholder/_4}}]]
2023-11-11 08:32:35.235350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [100000]
	 [[{{node Placeholder/_4}}]]


ValueError: in user code:

    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/opt/miniconda3/envs/hackathon/lib/python3.11/site-packages/keras/engine/compile

In [9]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [35]:
#user_input = input("Enter your input: ")
user_input1 = "archive/test/REAL/0079 (2).jpg"
user_input = 'archive/test/REAL/0318 (2).jpg'
#"sample_data/train_fake-1477_4.jpg"

In [40]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Preprocess the user input (assuming user_input is the input image file path)
img = image.load_img(user_input, target_size=(32, 32))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values

# Get model predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Model predictions:", predictions)
print(predictions[0][1])
print(predicted_class)

if predicted_class == 0:
    predicted_label = "real"
else:
    predicted_label = "fake"

    print(predicted_label)

1/1 [==============================] - 0s 22ms/step
Model predictions: [[0.40270525 0.59729475]]
0.59729475
1
fake


In [37]:
threshold = 0.99

# Get model predictions
predictions = model.predict(img_array)

# Check if the prediction score is greater than or equal to the threshold
if predictions >= threshold:
    predicted_class = "fake"
else:
    predicted_class = "real"

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 20ms/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [41]:
#print("Model predictions:", predictions)


Model predictions: [[0.40270525 0.59729475]]


In [25]:
# model.build(input_shape=(None, 32, 32, 3))
# model.summary()


# history = model.fit(
#   train_df,
#   validation_data=test_df,
#   epochs=5,
#   verbose=1
# )